In [12]:
import os
import glob
from nltk import sent_tokenize, word_tokenize
import spacy 
from collections import Counter

In [ ]:
nlp = spacy.load('en_core_web_lg')
nlp.max_length = nlp.max_length *1.5

def fx(s):
    return s[4:-4]

In [35]:
books = glob.glob('**/*.txt')
print(books)

['books\\01 - The Fellowship Of The Ring.txt', 'books\\02 - The Two Towers.txt', 'books\\03 - The Return Of The King.txt']


In [36]:
shelf = [{},{},{}]
for i,b in enumerate(books):
    with open(b) as f:
        t =  f.read()
        shelf[i] = {'title':fx(b), 'text': t,'n_words':len(t), 'n_sents': len(sent_tokenize(t))}
    print(fx(b), "\t--\t", shelf[i]['n_words'], shelf[i]['n_sents'])

s\01 - The Fellowship Of The Ring 	--	 1021113 13407
s\02 - The Two Towers 	--	 834790 12513
s\03 - The Return Of The King 	--	 724043 9139


In [4]:
for i,b in enumerate(shelf):
    x = nlp(b['text'])
    b['entities'] = x.ents

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


In [14]:
for b in shelf:
    b['characters'] = [e.text for e in b['entities'] if e.label_ == "PERSON"]
    print(len(b['characters']), len(b['entities']))

4362 8000
3625 6234
3362 6274


In [16]:
for b in shelf:
    tmp = Counter([wd for wd in word_tokenize(b['text']) if wd in b['characters']])
    b['mcc'] = tmp


In [28]:
for b in shelf:
    print(*[c[0][:7] for c in b['mcc'].most_common(13)],sep='\t')
    print(*[c[1] for c in b['mcc'].most_common(13)],sep='\t')
    print()

Frodo	Gandalf	Sam	Bilbo	Pippin	Aragorn	Strider	Merry	Boromir	Elrond	Tom	Gimli	Legolas
1095	463	400	323	209	206	204	190	154	134	126	120	95

Frodo	Sam	Gandalf	Gollum	Aragorn	Gimli	Pippin	Saruman	Legolas	Sméagol	Faramir	Merry	Treebea
495	418	339	307	289	192	192	176	175	133	133	131	130

Sam	Frodo	Gandalf	Pippin	Merry	Aragorn	Faramir	Denetho	Éomer	Rohan	Éowyn	Gimli	Beregon
459	388	300	281	222	221	158	120	104	95	92	76	72



In [33]:
b = shelf[0]['mcc'].most_common(20)
b2 = shelf[1]['mcc']
b3 = shelf[2]['mcc']
print(*[c[0][:7] for c in b],sep='\t')
print(*[c[1] for c in b],sep='\t')
print(*[b2[c[0]] for c in b],sep='\t')
print(*[b3[c[0]] for c in b],sep='\t')


Frodo	Gandalf	Sam	Bilbo	Pippin	Aragorn	Strider	Merry	Boromir	Elrond	Tom	Gimli	Legolas	Bree	Gollum	Sauron	Baggins	Lady	Saruman	Butterb
1095	463	400	323	209	206	204	190	154	134	126	120	95	73	68	60	58	54	52	51
495	339	418	8	192	289	0	131	88	13	2	192	175	2	307	34	6	24	176	0
388	300	459	32	281	221	0	222	27	33	6	76	67	19	22	47	9	48	41	21
